In [1]:
!pip install flair

     |████████████████████████████████| 61kB 4.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [47]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
import pandas as pd
import numpy as np

In [9]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/NLP/2_Competição/Dataset'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file= 'train.conll',
                              test_file= 'test.conll')

2021-05-12 19:20:17,335 Reading data from /content/drive/MyDrive/NLP/2_Competição/Dataset
2021-05-12 19:20:17,340 Train: /content/drive/MyDrive/NLP/2_Competição/Dataset/train.conll
2021-05-12 19:20:17,344 Dev: None
2021-05-12 19:20:17,347 Test: /content/drive/MyDrive/NLP/2_Competição/Dataset/test.conll


In [10]:
len(corpus.train)

6797

In [ ]:
test.head(10)

,AÇÃO
0,MONITÓRIA
1,.
2,PRESCRIÇÃO
3,DA
4,PRETENSÃO
5,MONITÓRIA
6,.
7,RECONHECIMENTO
8,.
9,MANUTENÇÃO


In [ ]:
-corpus.train[4]

Sentence: "UNÂNIME. , de acordo com a ata do julgamento e notas taquigráficas ."   [− Tokens: 13]

In [11]:
from flair.data import Corpus
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List

# 1. get the corpus
# corpus: Corpus = CONLL_03(base_path='/content/drive/MyDrive/NLP/2_Competição/Dataset/trainC.conll')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # GloVe embeddings
    WordEmbeddings('glove'),

    # # contextual string embeddings, forward
    # PooledFlairEmbeddings('news-forward', pooling='min'),

    # # contextual string embeddings, backward
    # PooledFlairEmbeddings('news-backward', pooling='min'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type)

# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/example-ner',
              train_with_dev=False,  
              max_epochs=2)


2021-05-12 19:21:10,144 ----------------------------------------------------------------------------------------------------
2021-05-12 19:21:10,147 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=17, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-05-12 19:21:10,151 ----------------------------------------------------------------------------------------------------
2021-05-12 19:21:10,155 Corpus: "Corpus: 6797 train + 755 dev + 2840 test sentences"
2021-05-12 19:21:10,157 ----------------------------------------------------------------------------------------------------
2021-05-12 19:21:10,159 Parameters:
2021-05-12 19:21:10,164  - learning

{'dev_loss_history': [],
 'dev_score_history': [],
 'test_score': 0.0,
 'train_loss_history': [19.522896779795825, 9.245672136040056]}

In [22]:
final = pd.read_csv("/content/resources/taggers/example-ner/test.tsv", sep = ' ', skip_blank_lines=False,header=None)

In [33]:
final2 = final

In [41]:
final2 = final2.drop(columns=[1,2])

In [42]:
final2['Tag'] = final[2]

In [51]:
final2 = final2.drop(columns=[0])

In [44]:
final2.index

RangeIndex(start=0, stop=14079, step=1)

In [49]:
final2["Id"] = pd.Series(np.arange(start=1, stop=14080, step=1), final2.index)

In [57]:
final3 = final2

In [58]:
final2 = final2.dropna()

In [59]:
final2

,Id,Tag
0,1,O
1,2,O
2,3,O
4,5,O
5,6,O
...,...,...
14072,14073,I-TEMPO
14073,14074,O
14075,14076,O
14076,14077,O


In [60]:
final2.to_csv('/content/drive/MyDrive/NLP/2_Competição/resultados/crime.csv',index=False)

In [55]:
cols = final2.columns.tolist()
print(cols)
cols = cols[-1:] + cols[:-1]
print(cols)
final2 = final2[cols]

['Tag', 'Id']
['Id', 'Tag']


In [18]:
def df_empty(columns, dtypes, index=None):
    assert len(columns)==len(dtypes)
    df = pd.DataFrame(index=index)
    for c,d in zip(columns, dtypes):
        df[c] = pd.Series(dtype=d)
    return df
    
df = df_empty(['a', 'b'])

TypeError: ignored

In [ ]:
from flair.data import Sentence
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('Arthur Galdino mora em Goiânia')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2021-05-12 16:53:42,948 loading file resources/taggers/example-ner/final-model.pt
Arthur <B-PESSOA> Galdino <I-PESSOA> mora <I-PESSOA> em Goiânia <B-LOCAL>


In [45]:
corpusTeste: Corpus = ColumnCorpus(data_folder, columns,
                                  train_file= 'trainC.conll',
                                  dev_file= 'testC.conll',
                                  test_file= 'test.conll')

2021-05-12 20:08:02,881 Reading data from /content/drive/MyDrive/NLP/2_Competição/Dataset
2021-05-12 20:08:02,883 Train: /content/drive/MyDrive/NLP/2_Competição/Dataset/trainC.conll
2021-05-12 20:08:02,885 Dev: /content/drive/MyDrive/NLP/2_Competição/Dataset/testC.conll
2021-05-12 20:08:02,887 Test: /content/drive/MyDrive/NLP/2_Competição/Dataset/test.conll


In [ ]:
saida = []


i=1

for text in corpusTeste.test:
  model.predict(text)
  for token in text:
    # print(i)
    tag = token.get_tag('ner')
    saida += ["".join(tag.value)]
    i = i+1
  i = i+1

In [ ]:
len(saida)

11239

In [ ]:
saida

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B - T E M P O',
 'I - T E M P O',
 'I - T E M P O',
 'I - T E M P O',
 'I - T E M P O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
ytest = pd.read_csv("/content/drive/MyDrive/NLP/2_Competição/Dataset/test.conll", sep = ' ', skip_blank_lines=False)

In [ ]:
acao = ytest.values

In [ ]:
import math
numbers = []
i=1

for index in acao:

  if type(index[0]) is not str:
    print("A")

A
A


array([['MONITÓRIA'],
       ['.'],
       [nan],
       ...,
       ['TST-RR-578.030/99.5'],
       ['\\'],
       [nan]], dtype=object)